In [1]:
import numpy as np
import scipy as sp
import sklearn as sk
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import seaborn as sns
import statsmodels as sm
import statsmodels.api as sm
import pandas as pd
from patsy import dmatrix

C:\Users\ledes\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = pd.read_csv("train.csv")

In [3]:
#for문 진행 사항 확인
from tqdm import tqdm_notebook
#역지오코딩(위도, 경도를 주소로 전환)하기 위한 모듈 geopy 의 클래스 중 'nominatim'
#Nominatim 사용 이유 : 개수 제한 문제를 해결할 수 있으므로
from geopy.geocoders import Nominatim
#역지오코딩 속도 개선을 위한 다중 쓰레드 사용
import threading
#지속적으로 많은 requests가 날아가면 "too many requests" 확인 -> 랜덤 시간 적용
import random
import time

In [4]:
#Nominatim 패키지 초기에 생긴 '접근 권한 없음' 에러 해결
#에러의 원인은 모르나, 해당 함수를 사용하면 역지오코딩은 성공적으로 가능
from geopy.geocoders import Nominatim
from urllib.request import Request

def get_geolocator():
    geolocator = Nominatim()

    requester = geolocator.urlopen

    def requester_hack(req, **kwargs):
        req = Request(url=req, headers=geolocator.headers)
        return requester(req, **kwargs)

    geolocator.urlopen = requester_hack

    return geolocator

In [5]:
#raw dataset에서 위도 및 경도 데이터 출력
pickup_lat = data["pickup_latitude"]
pickup_lon = data["pickup_longitude"]    

In [6]:
#쓰레드 사용 시, 순서가 섞이지 않게 다수의 데이터 프레임 생성 - 추후 합칠 예정
post_code = pd.DataFrame(columns = ["post_code1"])
post_code2 = pd.DataFrame(columns = ["post_code2"])
post_code3 = pd.DataFrame(columns = ["post_code3"])

In [8]:
def reverse_geocode() :
    
    for a, b in list(zip(pickup_lat, pickup_lon))[ : 100] :
        location = get_geolocator().reverse(str(a) + ',' + str(b), timeout = 10)
        address = location.address
        print("1")
        post_code.loc[len(post_code)] = address[-31 : -26]
        if len(post_code) % 10 == 0 :
            print(len(post_code))
        n = random.randint(1, 3)
        time.sleep(n)

def reverse_geocode2() :
    
    for a, b in list(zip(pickup_lat, pickup_lon))[200 : 300] :
        location = get_geolocator().reverse(str(a) + ',' + str(b), timeout = 10)
        address = location.address
        print("2")
        post_code2.loc[len(post_code)] = address[-31 : -26]
        if len(post_code) % 10 == 0 :
            print(len(post_code))
        n = random.randint(1, 3)
        time.sleep(n)
        
        
def reverse_geocode3() :
    
    for a, b in list(zip(pickup_lat, pickup_lon))[300 : 400] :
        location = get_geolocator().reverse(str(a) + ',' + str(b), timeout = 10)
        address = location.address
        print("3")
        post_code2.loc[len(post_code)] = address[-31 : -26]
        if len(post_code) % 10 == 0 :
            print(len(post_code))
        n = random.randint(1, 3)
        time.sleep(n)

In [ ]:
th1 = threading.Thread(target = reverse_geocode)
th1.demon  =True
th1.start()

th2 = threading.Thread(target = reverse_geocode2)
th2.demon = True
th2.start()

th3 = threading.Thread(target = reverse_geocode3)
th3.demon = True
th3.start()


## Nominatim error, 다른 클래스로 시도해볼 것.

In [ ]:
from geopy.geocoders import ArcGIS

In [ ]:
geolocator = ArcGIS()

In [ ]:
def reverse_geocode() :
    for a, b in list(zip(pickup_lat, pickup_lon))[ : 700000] :
        location = geolocator.reverse(str(a) + ',' + str(b), timeout = 10)
        address = location.address
        print("1")
        post_code.loc[len(post_code)] = address[-9 : -5]

def reverse_geocode2() :
    for a, b in list(zip(pickup_lat, pickup_lon))[700000 : ] :
        location = geolocator.reverse(str(a) + ',' + str(b), timeout = 10)
        address = location.address
        print("2")
        post_code2.loc[len(post_code)] = address[-9 : -5]

th1 = threading.Thread(target = reverse_geocode)
th1.start()

th2 = threading.Thread(target = reverse_geocode2)
th2.start()

In [ ]:
from geopy.geocoders import Yandex

In [ ]:
geolocator = Yandex()

In [ ]:
data.head()

In [ ]:
geolocator.reverse([40.76793, -73.982155 ])